We'll work with a dataset containing housing characteristics and skip the basic data set-up code.  The end result is:
- The housing characteristics for the training data are stored in a DataFrame `train_predictors`.  We'll use it to predict home prices in a Series called `target`.  
- The housing characteristics for the test data are stored in a DataFrame `test_predictors`.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read the data
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

# remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice

# drop columns with missing values (simplest approach)
cols_with_missing = [col for col in train_data.columns if train_data[col].isnull().any()]                                  
train_data.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1, inplace=True)
test_data.drop(['Id'] + cols_with_missing, axis=1, inplace=True)

# "cardinality" means the number of unique values in a column
# select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in train_data.columns if
                        train_data[cname].nunique() < 10 and
                        train_data[cname].dtype == "object"]

# select numeric columns
numeric_cols = [cname for cname in train_data.columns if
                train_data[cname].dtype in ['int64', 'float64']]

# keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X = train_data[my_cols]
X_test = test_data[my_cols]

# break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,
                                                      random_state=0)

In [ ]:
X_train.head()

In [ ]:
# get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# apply label encoder to each column with categorical data
my_encoder = LabelEncoder()
for col in object_cols:
    my_encoder.fit(X[col])
    label_X_train[col] = my_encoder.transform(X_train[col])
    label_X_valid[col] = my_encoder.transform(X_valid[col])

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
# one-hot encode categorical data
OH_X_train = pd.get_dummies(X_train)
OH_X_valid = pd.get_dummies(X_valid)

# ensure columns are in same order in both datasets
OH_X_train, OH_X_valid = OH_X_train.align(OH_X_valid, join='inner', axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

from ex1 // imputation + encoding

In [ ]:
# "cardinality" means the number of unique values in a column
# select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in train_data.columns if
                        train_data[cname].nunique() < 10 and
                        train_data[cname].dtype == "object"]

# select numeric columns
numeric_cols = [cname for cname in train_data.columns if
                train_data[cname].dtype in ['int64', 'float64']]

# keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X = train_data[my_cols]
X_test_full = test_data[my_cols]

# break off validation set from training data
X_train_full, X_valid_full, y_train_full, y_valid_full = train_test_split(X, y, 
                                                                          train_size=0.7, test_size=0.3,
                                                                          random_state=0)

In [ ]:
X_train_full.head()